In [4]:
import pandas as pd
import torch, torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint

In [5]:
df = pd.read_csv('data.csv')
df1 = df[df['label']=='time']

In [6]:
df1

,label,T [degC],P [bar],W_cat [g],VL [mL],Time [h],C_Gly [M],C_KOH [M],C_H2O [M],C_H2 [M],C_LA [M],C_PDO [M],C_FA [M],C_GA [M],C_ME [M],C_EG [M]
0,time,200,15.518495,0.3,41.253968,0.0,6.060023,9.090035,29.903412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,time,200,60.025635,0.3,41.253968,0.5,2.860331,6.085091,32.960788,0.110246,2.765253,0.292123,0.131612,0.108078,0.022471,0.022471
2,time,200,83.186841,0.3,41.253968,1.0,1.084744,4.480322,34.634980,0.144885,4.223836,0.507732,0.198998,0.186878,0.050773,0.050773
3,time,200,88.384487,0.3,41.253968,1.5,0.727203,4.142026,34.995725,0.152584,4.563197,0.529116,0.198466,0.186346,0.048102,0.048102
4,time,200,94.400527,0.3,41.253968,2.0,0.339361,3.764094,35.335154,0.161461,4.878318,0.553424,0.232901,0.214721,0.065109,0.065109
5,time,200,94.084084,0.3,41.253968,2.5,0.121200,3.664676,35.544967,0.160995,4.938919,0.702635,0.261400,0.225040,0.054048,0.054048


In [18]:
# parameters initialization
k = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.float32)

In [14]:
# Concentration initialization
C0 = torch.tensor([
    df1.iloc[0]['C_Gly [M]'],    # 글리세롤
    df1.iloc[0]['C_KOH [M]'],    # KOH
    df1.iloc[0]['C_H2 [M]'],     # 수소
    df1.iloc[0]['C_LA [M]'],     # 젖산
    df1.iloc[0]['C_PDO [M]'],    # 1,3-PDO
    df1.iloc[0]['C_FA [M]'],     # 포름산
    df1.iloc[0]['C_GA [M]'],     # 글리콜산
    df1.iloc[0]['C_ME [M]'],     # 메탄올
    df1.iloc[0]['C_EG [M]']      # 에틸렌글리콜
], dtype=torch.float32)

C_exp = torch.tensor([
    df1.iloc[1:]['C_Gly [M]'].values,    # 글리세롤 0
    df1.iloc[1:]['C_KOH [M]'].values,    # KOH 1
    df1.iloc[1:]['C_H2 [M]'].values,     # 수소 2
    df1.iloc[1:]['C_LA [M]'].values,     # 젖산 3
    df1.iloc[1:]['C_PDO [M]'].values,    # 1,3-PDO 4
    df1.iloc[1:]['C_FA [M]'].values,     # 포름산 5
    df1.iloc[1:]['C_GA [M]'].values,     # 글리콜산 6
    df1.iloc[1:]['C_ME [M]'].values,     # 메탄올 7 
    df1.iloc[1:]['C_EG [M]'].values      # 에틸렌글리콜 8
], dtype=torch.float32)


C:\Users\USER\AppData\Local\Temp\ipykernel_51424\4040965680.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  C_exp = torch.tensor([


In [10]:
class ParamNet(nn.Module):
    def __init__(self, in_dim, hidden, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.SiLU(),
            nn.Linear(hidden, hidden),
            nn.SiLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return torch.pow(10.0, self.net(x))



In [17]:
def MB(t, C, k):
    
    wcat = 0.3*0.02 # Pt 함량 0.3 g의 2%

    # kg1 0 # KGly 1 # KH2 2 # KPDO 3 # kd1 4 # kd2 5 # kd3 6 # kp1 7 # kp2 8

    # 글리세롤 0 # KOH 1 # 수소 2 # 젖산 3 # 1,3-PDO 4 # 포름산 5 # 글리콜산 6 # 메탄올 7 # 에틸렌글리콜 8
    
    theta = 1 + k[1]*C[0] + (k[2]*C[2])**0.5 + k[3]*C[4]
    rg1= k[0]*k[1]*C[0]/(theta**3)*wcat # Glycerol -> DHA + H2
    rd1 = k[4]*wcat # DHA -> PRA + H2O
    rd2 = k[5]*(C[1]**2)*wcat # DHA + 2KOH -> FA + GA + 2H2
    rd3 = k[6]*(C[2]**2)*wcat # DHA + 2H2 -> ME + EG
    rp1 = k[7]*C[1]*wcat # PRA + KOH -> LA
    rp2 = k[8]*k[2]*C[2]/(theta**3)*wcat # PRA + H2 -> PDO

    fd1 = rd1/(rd1+rd2+rd3)
    fd2 = rd2/(rd1+rd2+rd3)
    fd3 = rd3/(rd1+rd2+rd3)

    fp1 = rp1/(rp1+rp2)
    fp2 = rp2/(rp1+rp2)

    dC0dt = -rg1 # 
    dC1dt = -2*rg1*fd2 - rg1*fd1*fp1
    dC2dt = rg1 + 2*rg1*fd2 - 2*rg1*fd3 - rg1*fd1*fp2
    dC3dt = rg1*fd1*fp1
    dC4dt = rg1*fd1*fp2
    dC5dt = rg1*fd2
    dC6dt = rg1*fd2
    dC7dt = rg1*fd3
    dC8dt = rg1*fd3

    return torch.stack([dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dC5dt, dC6dt, dC7dt, dC8dt])


SyntaxError: invalid syntax (3340244734.py, line 16)